## Connect to the NHANES data resource using the HPDS Adapter

In [1]:
import PicSureHpdsLib
adapter = PicSureHpdsLib.BypassAdapter("http://pic-sure-hpds-nhanes:8080/PIC-SURE")
resource = adapter.useResource()

### A "resource" allows access to the Data Dictionary and Query Engine of the underlying  data source

In [2]:
resource.help()


        [HELP] PicSureHpdsLib.useResource(resource_uuid)
            .dictionary()       Used to access data dictionary of the resource
            .query()            Used to query against data in the resource
            
        [ENVIRONMENT]
              Endpoint URL: http://pic-sure-hpds-nhanes:8080/PIC-SURE/
             Resource UUID: None


## Create a new query instance and use .help() to see what can be done with it

In [3]:
query = resource.query()
query.help()


        .select()   list of data fields to return from resource for each record
        .require()  list of data fields that must be present in all returned records
        .filter()   list of data fields and conditions that returned records satisfy
                  [ Filter keys exert an AND relationship on returned records      ]
                  [ Categorical values have an OR relationship on their key        ]
                  [ Numerical Ranges are inclusive of their start and end points   ]

        .getCount()             returns a count indicating the number of matching numbers
        .getResults()           returns a CSV-like string containing the matching records
        .getResultsDataFrame()  returns a pandas DataFrame containing the matching records
        .getRunDetails()        returns details about the last run of the query
        .getQueryCommand()      returns the JSON-formatted query request
        .show()                 lists all current query parameters
  

### Use .help() to see how to add entries to the selection criteria

In [4]:
query.select().help()


            select().
              add("key")            add a single column to be returned in results
              add(["key1", "key2"]) add several columns to be returned in results
              delete("key")         delete a single column from the list of columns to return
              show()                lists all current columns that will be returned in results
              clear()               clears all values from the select list
            


### Use a chained of commands on the dictionary object to include all labs in the query's selection list

In [5]:
query.select().add(resource.dictionary().find("laboratory").keys())

### Use the .show() command to see what is currently selected for the query

In [7]:
query.show()

.__________[ Query.Select()  Settings ]_____________________________________________________________________________________________________________________
| _key__________________________________________________________________________________________________________________________
|  \\laboratory\\bacterial infection\\Imipenem 1\\                                                                                  |
|  \\laboratory\\pcbs\\PCB156 (ng per g)\\                                                                                          |
|  \\laboratory\\polybrominated ethers\\2,2,4,4,5,6-hexabromodiphenyl ether\\                                                       |
|  \\laboratory\\biochemistry\\Creatinine, urine (umol per L)\\14763\\                                                              |
|  \\laboratory\\pesticides\\Oxychlordane (ng per g)\\                                                                              |
|  \\laboratory\\biochemistry\\Creatinine, u

### Clear the current selection list and add only "\\laboratory\\hormone\\" results

In [6]:
query.select().clear()
query.select().add(resource.dictionary().find("\\laboratory\\hormone\\").keys())
query.select().show()

cleared list
| _key__________________________________________________________________________________________________________________________
|  \\laboratory\\hormone\\Insulin: SI(pmol per L)\\                                                                                 |
|  \\laboratory\\hormone\\Follicle stimulating hormone (mIU per mL)\\                                                               |
|  \\laboratory\\hormone\\Parathyroid Hormone(Elecys method) pg per mL\\                                                            |
|  \\laboratory\\hormone\\Thyroxine (T4) (ug per dL)\\                                                                              |
|  \\laboratory\\hormone\\Luteinizing hormone (mIU per mL)\\                                                                        |
|  \\laboratory\\hormone\\                                                                                                          |
|  \\laboratory\\hormone\\Insulin (uU per mL)\\       

### Run some commands on the dictionary to find and then filter the query by gender 

In [7]:
gender = resource.dictionary().find("sex")
gender.help()


        [HELP] PicSureHpdsLib.Client(connection).useResource(uuid).dictionary().find(term)
            .count()        Returns the number of entries in the dictionary that match the given term
            .keys()         Return the keys of the matching entries
            .entries()      Return a list of matching dictionary entries
            .DataFrame()    Return the entries in a Pandas-compatible format
             
        [Examples]
            results = PicSureHpdsLib.Client(connection).useResource(uuid).dictionary().find("asthma")
            df = results.DataFrame()
        


In [8]:
gender.DataFrame()

HpdsDataType  categorical  \
\questionnaire\sexual behavior\Ever had sexual ...   phenotypes         True   
\questionnaire\sexual behavior\Are you circumci...   phenotypes         True   
\questionnaire\sexual behavior\                      phenotypes        False   
\demographics\SEX\                                   phenotypes         True   

                                                    categoryValues      max  \
\questionnaire\sexual behavior\Ever had sexual ...       [No, Yes]      NaN   
\questionnaire\sexual behavior\Are you circumci...       [No, Yes]      NaN   
\questionnaire\sexual behavior\                               None  17040.0   
\demographics\SEX\                                  [female, male]      NaN   

                                                    min  observationCount  
\questionnaire\sexual behavior\Ever had sexual ...  NaN              8271  
\questionnaire\sexual behavior\Are you circumci...  NaN              5178  
\questionnaire\sexual behavior\                     0.0             41474  
\demographics\SEX\                                  NaN             41474

### Add a filter on our query to filter results to only female subjects

In [9]:
query.filter().help()


            filter().
              add("key", value)                  - or -
              add("key", "value")               filter to records with KEY column that equals VALUE
              add("key", ["value1", "value2"])  filter to records with KEY column equalling one value within the given list
              add("key", start, end)            filter to records with KEY column value between START and END (inclusive)
                                                    start -or- end may be set to None to filter by a max or min value
              delete("key")                     delete a filter from the list of filters
              show()                            lists all current filters that results records must satisfy
              clear()                           clears all values from the filters list
            


In [10]:
query.filter().add("\\demographics\\SEX\\", ["female"])

## View our query's criterion as it currently stands

In [11]:
query.show()

.__________[ Query.Select()  Settings ]_____________________________________________________________________________________________________________________
| _key__________________________________________________________________________________________________________________________
|  \\laboratory\\hormone\\Insulin: SI(pmol per L)\\                                                                                 |
|  \\laboratory\\hormone\\Follicle stimulating hormone (mIU per mL)\\                                                               |
|  \\laboratory\\hormone\\Parathyroid Hormone(Elecys method) pg per mL\\                                                            |
|  \\laboratory\\hormone\\Thyroxine (T4) (ug per dL)\\                                                                              |
|  \\laboratory\\hormone\\Luteinizing hormone (mIU per mL)\\                                                                        |
|  \\laboratory\\hormone\\                  

### Run the query and display the results

In [12]:
query.help()


        .select()   list of data fields to return from resource for each record
        .require()  list of data fields that must be present in all returned records
        .filter()   list of data fields and conditions that returned records satisfy
                  [ Filter keys exert an AND relationship on returned records      ]
                  [ Categorical values have an OR relationship on their key        ]
                  [ Numerical Ranges are inclusive of their start and end points   ]

        .getCount()             returns a count indicating the number of matching numbers
        .getResults()           returns a CSV-like string containing the matching records
        .getResultsDataFrame()  returns a pandas DataFrame containing the matching records
        .getRunDetails()        returns details about the last run of the query
        .getQueryCommand()      returns the JSON-formatted query request
        .show()                 lists all current query parameters
  

In [13]:
query.getCount()

[ERROR] could not convert results of RequestCount to integer


'21210'

In [14]:
query.getResultsDataFrame()

Patient ID \demographics\SEX\  \laboratory\hormone\  \
0               3             female                   NaN   
1               4             female                   NaN   
2               6             female                   NaN   
3               8             female                   NaN   
4              11             female                   NaN   
5              12             female                   NaN   
6              13             female                  0.90   
7              14             female                   NaN   
8              15             female                   NaN   
9              16             female                   NaN   
10             17             female                   NaN   
11             19             female                   NaN   
12             20             female                   NaN   
13             21             female                   NaN   
14             24             female                   NaN   
15             25             female                   NaN   
16             28             female                   NaN   
17             30             female                   NaN   
18             31             female                   NaN   
19             32             female                   NaN   
20             33             female                  0.30   
21             39             female                   NaN   
22             43             female                   NaN   
23             45             female                   NaN   
24             46             female                   NaN   
25             48             female                   NaN   
26             49             female                   NaN   
27             50             female                  0.14   
28             52             female                   NaN   
29             53             female                   NaN   
...           ...                ...                   ...   
21180       41418             female                   NaN   
21181       41422             female                   NaN   
21182       41425             female                   NaN   
21183       41427             female                   NaN   
21184       41430             female                   NaN   
21185       41431             female                   NaN   
21186       41432             female                   NaN   
21187       41435             female                   NaN   
21188       41438             female                  0.30   
21189       41439             female                   NaN   
21190       41440             female                   NaN   
21191       41442             female                   NaN   
21192       41443             female                   NaN   
21193       41445             female                   NaN   
21194       41447             female                   NaN   
21195       41450             female                   NaN   
21196       41451             female                   NaN   
21197       41454             female                   NaN   
21198       41455             female                   NaN   
21199       41457             female                  0.14   
21200       41458             female                   NaN   
21201       41459             female                   NaN   
21202       41461             female                   NaN   
21203       41464             female                   NaN   
21204       41466             female                   NaN   
21205       41467             female                   NaN   
21206       41468             female                  0.14   
21207       41469             female                  0.00   
21208       41470             female                  0.00   
21209       41472             female                  0.00   

       \laboratory\hormone\Follicle stimulating hormone (mIU per mL)\  \
0                                                    NaN                
1               

# You can even work with multiple queries at the same time without having them accidently interact with each other!

In [15]:
query2 = resource.query()
query3 = resource.query()


query2.select().add(resource.dictionary().find("\\demographics\\AGE").keys())
query3.select().add(resource.dictionary().find("\\demographics\\AGE").keys())


query2.filter().add(resource.dictionary().find("\\demographics\\AGE").keys(), min=60)
query3.filter().add(resource.dictionary().find("\\demographics\\AGE").keys(), max=20)
#query2.filter().add(resource.dictionary().find("\\demographics\\AGE").keys(), 60, None)
#query3.filter().add(resource.dictionary().find("\\demographics\\AGE").keys(), None, 20)

too_old = query2.getResultsDataFrame()
too_young = query3.getResultsDataFrame()

In [16]:
too_young.count()

Patient ID            21163
\demographics\AGE\    21163
dtype: int64

In [17]:
too_old.count()

Patient ID            7177
\demographics\AGE\    7177
dtype: int64